In [1]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from langchain.tools import DuckDuckGoSearchResults


llm = ChatOpenAI(temperature=0.1)

class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Research about the xz backdoor"
    )
class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Use this tool to search for information on the xz backdoor.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        SearchToolArgsSchema
    ] = SearchToolArgsSchema

    def _run(self, query):
        wiki = WikipediaAPIWrapper()
        results= wiki.run(query)
        with open('output.txt', 'w') as file:
            file.write("Wikipedia Results:\n" + results + "\n\n")
        return results

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    Use this tool to search for information on the xz backdoor.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        SearchToolArgsSchema
    ] = SearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        results = ddg.run(query)
        with open('output.txt', 'a') as file:
            file.write("DuckDuckGo Results:\n" + results + "\n\n")
        return results


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool()
    ],
)

prompt = "Research about the xz backdoor in wikipedia and duckduckgo sites"

output = agent.invoke(prompt)




> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'xz backdoor'}`


Page: XZ Utils backdoor
Summary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, the backdoor only targeted Debian- and RPM-based systems running on the x86-64 architecture. At the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been assigned a CVSS score of 10.0, the highest possible score.

Page: XZ Utils
Summary: XZ Utils (previously LZMA Utils) is a set of free software command-line lossless da

HTTPError: 